In [ ]:
#!conda install gdcm -c conda-forge -y

import os

from PIL import Image
import pandas as pd
from tqdm.auto import tqdm

In [ ]:
import os
from PIL import Image
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
import pydicom
import glob

In [ ]:

from pydicom.pixel_data_handlers.util import apply_voi_lut

def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data

In [ ]:
def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    
    return im

In [ ]:
train_df = pd.read_csv('../input/siim-covid19-detection/train_image_level.csv')
train_df.head(2)

In [ ]:
#load the identified images to be dropped from previous notebook ljmu-2-preprocessing
drop_df = pd.read_csv('../input/images-to-drop/images_to_drop.csv')
drop_df.head(3)

In [ ]:
# mark the rows for dropping in train_df if that image is present in drop_df
# and convert remaining dicoms to jpg
#train_df['drop'] = train_df.apply(lambda row: 'Y' if (row['id'] in drop_df['p1'].values) else 'N', axis=1)
#train_df[train_df['drop']=='Y'].head(3)

In [ ]:
path = '../input/siim-covid19-detection/train/00086460a852/9e8302230c91/65761e66de9f.dcm'
dicom = pydicom.read_file(path)
print(dicom)

In [ ]:
base_path = '../input/siim-covid19-detection'
train_files = sorted(glob.glob(os.path.join(base_path, 'train/*/*/*.dcm')))
print(f'Number of training files: {len(train_files)}')

In [ ]:
image_id = []
dim0 = []
dim1 = []
splits = []
images_to_drop = drop_df['p1'].tolist()

save_dir = f'/kaggle/tmp/train/'
split='train'
os.makedirs(save_dir, exist_ok=True)
#os.walk(f'../input/siim-covid19-detection/train')
for path in tqdm(train_files):
    # set keep_ratio=True to have original aspect ratio
    file = path.split('/')[-1]
    if(file.replace('.dcm', '_image') in images_to_drop):
        continue
    xray = read_xray(path)
    im = resize(xray, size=256)  
    im.save(os.path.join(save_dir, file.replace('.dcm', '.jpg')))

    image_id.append(file.replace('.dcm', ''))
    dim0.append(xray.shape[0])
    dim1.append(xray.shape[1])
    splits.append(split)

In [ ]:
%%time
!tar -zcf train.tar.gz -C "/kaggle/tmp/train/" .


In [ ]:
df = pd.DataFrame.from_dict({'image_id': image_id, 'dim0': dim0, 'dim1': dim1, 'split': splits})
df.to_csv('meta.csv', index=False)

In [ ]:
# check duplicates between 
import pandas as pd
img_df = pd.read_csv('../input/siim-covid19-detection/train_image_level.csv')
stu_df = pd.read_csv('../input/siim-covid19-detection/train_study_level.csv')
img_df.head(1)

In [ ]:
stu_df.head(1)

In [ ]:
lis1 = img_df['StudyInstanceUID'].tolist()
lis2 = [l.replace("_study","") for l in stu_df['id'].tolist() ]
print( lis1[0] + "  " + lis2[0])

In [ ]:
s1 = set(lis1)
s2 = set(lis2)

In [ ]:
# unique entries in image set not in study set
print(s1.difference(s2))

In [ ]:
# unique entries in study set not in image set
print(s2.difference(s1))

All images are converted to jpg from dicom.